# End to end

In [1]:
import sagemaker
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

prefix = 'sagemaker/spark-preprocess-demo/' + timestamp_prefix
input_prefix = prefix + '/input/raw/abalone'
input_preprocessed_prefix = prefix + '/input/preprocessed/abalone'
model_prefix = prefix + '/model'
#Jay Change
mleap_model_prefix = prefix + '/mleap-model'

In [2]:
# Fetch the dataset from the SageMaker bucket
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv

# Uploading the training data to S3
sagemaker_session.upload_data(path='abalone.csv', bucket=bucket, key_prefix=input_prefix)

--2020-06-12 18:31:52--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.200.80
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.200.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [binary/octet-stream]
Saving to: ‘abalone.csv’

abalone.csv         100%[===================>] 187.38K   824KB/s    in 0.2s    

2020-06-12 18:31:53 (824 KB/s) - ‘abalone.csv’ saved [191873/191873]



's3://sagemaker-us-east-1-328296961357/sagemaker/spark-preprocess-demo/2020-06-12-18-31-52/input/raw/abalone/abalone.csv'

In [3]:
%cd container
!docker build -t sagemaker-spark-example .
%cd ../

/home/ec2-user/SageMaker/airflow-sagemaker/end-to-end-pipeline/container
Sending build context to Docker daemon  17.36MB
Step 1/34 : FROM openjdk:8-jre-slim
 ---> 778bc46b8d12
Step 2/34 : RUN apt-get update
 ---> Using cache
 ---> 7098adcac4c9
Step 3/34 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> f046d58588a5
Step 4/34 : RUN pip3 install py4j psutil==5.6.5 mleap==0.8.1 boto3
 ---> Using cache
 ---> 663ef77ac837
Step 5/34 : RUN apt-get clean
 ---> Using cache
 ---> 2f65a6afff2e
Step 6/34 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> f61c8c50496b
Step 7/34 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> ba8a91d8e91b
Step 8/34 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 1a871870b26b
Step 9/34 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> b755367bdca0
Step 10/34 : ENV HADOOP_VERSION 3.0.0
 ---> Using cache
 ---> 3cb2d40cbc79
Step 11/34 : ENV HADOOP_HOME /usr/hadoo

In [4]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'sagemaker-spark-example'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
spark_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $spark_repository_uri
!docker push $spark_repository_uri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-spark-example' already exists in the registry with id '328296961357'
The push refers to repository [328296961357.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark-example]

edbf978f: Preparing 
d04b810b: Preparing 
c5c3c59f: Preparing 
1506440b: Preparing 
b377677a: Preparing 
b24cd1fc: Preparing 
9681573c: Preparing 
82056577: Preparing 
a88de2cc: Preparing 
87d8bf9e: Preparing 
628f2670: Preparing 
665fdbe9: Preparing 
cd2ee037: Preparing 
386e5425: Preparing 
b21953f4: Layer already exists Klatest: digest: sha256:2c61b6939352ca9c924ebddeebea22

In [5]:
%%writefile preprocess.py
from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import boto3
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructField, StructType, StringType, DoubleType
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from mleap.pyspark.spark_support import SimpleSparkSerializer

def csv_line(data):
    r = ','.join(str(d) for d in data[1])
    return str(data[0]) + "," + r


def main():
    spark = SparkSession.builder.appName("PySparkAbalone").getOrCreate()
    
    # Convert command line args into a map of args
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    spark.sparkContext._jsc.hadoopConfiguration().set("mapred.output.committer.class",
                                                      "org.apache.hadoop.mapred.FileOutputCommitter")
    
    # Defining the schema corresponding to the input data. The input data does not contain the headers
    schema = StructType([StructField("sex", StringType(), True), 
                         StructField("length", DoubleType(), True),
                         StructField("diameter", DoubleType(), True),
                         StructField("height", DoubleType(), True),
                         StructField("whole_weight", DoubleType(), True),
                         StructField("shucked_weight", DoubleType(), True),
                         StructField("viscera_weight", DoubleType(), True), 
                         StructField("shell_weight", DoubleType(), True), 
                         StructField("rings", DoubleType(), True)])

    # Downloading the data from S3 into a Dataframe
    total_df = spark.read.csv(('s3a://' + os.path.join(args['s3_input_bucket'], args['s3_input_key_prefix'],
                                                   'abalone.csv')), header=False, schema=schema)

    #StringIndexer on the sex column which has categorical value
    sex_indexer = StringIndexer(inputCol="sex", outputCol="indexed_sex")
    
    #one-hot-encoding is being performed on the string-indexed sex column (indexed_sex)
    sex_encoder = OneHotEncoder(inputCol="indexed_sex", outputCol="sex_vec")

    #vector-assembler will bring all the features to a 1D vector for us to save easily into CSV format
    assembler = VectorAssembler(inputCols=["sex_vec", 
                                           "length", 
                                           "diameter", 
                                           "height", 
                                           "whole_weight", 
                                           "shucked_weight", 
                                           "viscera_weight", 
                                           "shell_weight"], 
                                outputCol="features")
    
    # The pipeline comprises of the steps added above
    pipeline = Pipeline(stages=[sex_indexer, sex_encoder, assembler])
    
    # This step trains the feature transformers
    model = pipeline.fit(total_df)
    
    # This step transforms the dataset with information obtained from the previous fit
    transformed_total_df = model.transform(total_df)
    
    # Split the overall dataset into 80-20 training and validation
    (train_df, validation_df) = transformed_total_df.randomSplit([0.8, 0.2])
    
    # Convert the train dataframe to RDD to save in CSV format and upload to S3
    train_rdd = train_df.rdd.map(lambda x: (x.rings, x.features))
    train_lines = train_rdd.map(csv_line)
    train_lines.saveAsTextFile('s3a://' + os.path.join(args['s3_output_bucket'], args['s3_output_key_prefix'], 'train'))
    
    # Convert the validation dataframe to RDD to save in CSV format and upload to S3
    validation_rdd = validation_df.rdd.map(lambda x: (x.rings, x.features))
    validation_lines = validation_rdd.map(csv_line)
    validation_lines.saveAsTextFile('s3a://' + os.path.join(args['s3_output_bucket'], args['s3_output_key_prefix'], 'validation'))
    
    # Serialize and store the model via MLeap  
    SimpleSparkSerializer().serializeToBundle(model, "jar:file:/opt/ml/model.zip", validation_df)    
    # Unzip the model as SageMaker expects a .tar.gz file but MLeap produces a .zip file
    import zipfile
    with zipfile.ZipFile("/opt/ml/model.zip") as zf:
        zf.extractall("/opt/ml/model")

    # Writw back the content as a .tar.gz file
    import tarfile
    with tarfile.open("/opt/ml/model.tar.gz", "w:gz") as tar:
        tar.add("/opt/ml/model/bundle.json", arcname='bundle.json')
        tar.add("/opt/ml/model/root", arcname='root')
    
    # Upload the model in tar.gz format to S3 so that it can be used with SageMaker for inference later
    s3 = boto3.resource('s3') 
    file_name = os.path.join(args['s3_mleap_model_prefix'], 'model.tar.gz')
    s3.Bucket(args['s3_model_bucket']).upload_file('/opt/ml/model.tar.gz', file_name)    

if __name__ == "__main__":
    main()

Writing preprocess.py


In [6]:
from sagemaker.processing import ScriptProcessor, ProcessingInput
spark_processor = ScriptProcessor(base_job_name='spark-preprocessor',
                                  image_uri=spark_repository_uri,
                                  command=['/opt/program/submit'],
                                  role=role,
                                  instance_count=2,
                                  instance_type='ml.r5.xlarge',
                                  max_runtime_in_seconds=1200,
                                  env={'mode': 'python'})

spark_processor.run(code='preprocess.py',
                    arguments=['s3_input_bucket', bucket,
                               's3_input_key_prefix', input_prefix,
                               's3_output_bucket', bucket,
                               's3_output_key_prefix', input_preprocessed_prefix,
                               's3_model_bucket', bucket,
                               's3_mleap_model_prefix', mleap_model_prefix],
                    logs=True)


Job Name:  spark-preprocessor-2020-06-12-18-31-56-434
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-328296961357/spark-preprocessor-2020-06-12-18-31-56-434/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []
2020-06-12 18:35:27,248 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.0.71.156
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.0.0
STARTUP_MSG:   classpath = /usr/hadoop-3.0.0/etc/hadoop:/usr/hadoop-3.0.0/share/hadoop/common/lib/jsr305-3.0.0.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/metrics-core-3.0.1.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/gson-2.2.4.jar:/usr/hadoop-3.0.0/share/hadoop/common/lib/commons-net-3.1.jar:/usr/hadoop-3.0.0/share/hadoop/co

Starting resourcemanager
Starting nodemanagers
localhost: /usr/hadoop-3.0.0/bin/../libexec/hadoop-functions.sh: line 981: ssh: command not found
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/spark-2.2.0/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
ml.combust.mleap#mleap-spark_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent;1.0
#011confs: [default]
#011found ml.combust.mleap#mleap-spark_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-spark-base_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-runtime_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-core_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-base_2.11;0.8.1 in central
#011found ml.combust.mleap#mleap-tensor_2.11;0.8.1 in central
#011found io.spray#spray-json_2.11;1.3.2 in central
#011found org.apache.spark#spark-mllib-local_2.11;2.2.0 in central

2020-06-12 18:35:53,753 INFO yarn.Client: Application report for application_1591986937479_0001 (state: ACCEPTED)
2020-06-12 18:35:54,755 INFO yarn.Client: Application report for application_1591986937479_0001 (state: ACCEPTED)
2020-06-12 18:35:55,758 INFO yarn.Client: Application report for application_1591986937479_0001 (state: ACCEPTED)
2020-06-12 18:35:56,761 INFO yarn.Client: Application report for application_1591986937479_0001 (state: ACCEPTED)
2020-06-12 18:35:57,690 INFO cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: ApplicationMaster registered as NettyRpcEndpointRef(spark-client://YarnAM)
2020-06-12 18:35:57,694 INFO cluster.YarnClientSchedulerBackend: Add WebUI Filter. org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter, Map(PROXY_HOSTS -> algo-1, PROXY_URI_BASES -> http://algo-1:8088/proxy/application_1591986937479_0001), /proxy/application_1591986937479_0001
2020-06-12 18:35:57,695 INFO ui.JettyUtils: Adding filter: org.apache.hadoop.yarn.server.webproxy.amfil

2020-06-12 18:36:27,126 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 2.0 (TID 2) in 4300 ms on algo-2 (executor 1) (1/1)
2020-06-12 18:36:27,127 INFO cluster.YarnScheduler: Removed TaskSet 2.0, whose tasks have all completed, from pool 
2020-06-12 18:36:27,128 INFO scheduler.DAGScheduler: ResultStage 2 (saveAsTextFile at NativeMethodAccessorImpl.java:0) finished in 4.302 s
2020-06-12 18:36:27,129 INFO scheduler.DAGScheduler: Job 1 finished: saveAsTextFile at NativeMethodAccessorImpl.java:0, took 4.328914 s
2020-06-12 18:36:27,987 INFO datasources.FileSourceStrategy: Pruning directories with: 
2020-06-12 18:36:27,987 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2020-06-12 18:36:27,988 INFO datasources.FileSourceStrategy: Output Data Schema: struct<sex: string, length: double, diameter: double, height: double, whole_weight: double ... 7 more fields>
2020-06-12 18:36:27,988 INFO execution.FileSourceScanExec: Pushed Filters: 
2020-06-12 18:36:28,078 INFO codegen.Co

2020-06-12 18:36:34
Finished Yarn configuration files setup.

Received end of job signal, exiting...
Finished Yarn configuration files setup.




In [7]:
print('Top 5 rows from s3://{}/{}/train/'.format(bucket, input_preprocessed_prefix))
!aws s3 cp --quiet s3://$bucket/$input_preprocessed_prefix/train/part-00000 - | head -n5

Top 5 rows from s3://sagemaker-us-east-1-328296961357/sagemaker/spark-preprocess-demo/2020-06-12-18-31-52/input/preprocessed/abalone/train/
5.0,0.0,0.0,0.29,0.225,0.075,0.14,0.0515,0.0235,0.04
7.0,0.0,0.0,0.305,0.225,0.07,0.1485,0.0585,0.0335,0.045
7.0,0.0,0.0,0.325,0.26,0.09,0.1915,0.085,0.036,0.062
9.0,0.0,0.0,0.33,0.26,0.08,0.2,0.0625,0.05,0.07
6.0,0.0,0.0,0.335,0.22,0.07,0.17,0.076,0.0365,0.05


In [8]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sagemaker_session.boto_region_name, 'xgboost', repo_version="0.90-1")
print(training_image)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3


In [9]:
s3_train_data = 's3://{}/{}/{}'.format(bucket, input_preprocessed_prefix, 'train/part')
s3_validation_data = 's3://{}/{}/{}'.format(bucket, input_preprocessed_prefix, 'validation/part')
s3_output_location = 's3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

xgb_model = sagemaker.estimator.Estimator(training_image,
                                          role, 
                                          train_instance_count=1, 
                                          train_instance_type='ml.m4.xlarge',
                                          train_volume_size = 20,
                                          train_max_run = 3600,
                                          input_mode= 'File',
                                          output_path=s3_output_location,
                                          sagemaker_session=sagemaker_session)

xgb_model.set_hyperparameters(objective = "reg:linear",
                              eta = .2,
                              gamma = 4,
                              max_depth = 5,
                              num_round = 10,
                              subsample = 0.7,
                              silent = 0,
                              min_child_weight = 6)

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/csv', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/csv', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

In [10]:
xgb_model.fit(inputs=data_channels, logs=True)

2020-06-12 18:38:12 Starting - Starting the training job......
2020-06-12 18:38:46 Starting - Launching requested ML instances......
2020-06-12 18:40:07 Starting - Preparing the instances for training......
2020-06-12 18:41:00 Downloading - Downloading input data...
2020-06-12 18:41:31 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[18:41:53] 3353x9 matrix with 30177 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV

In [23]:
s3_output_location + '/sagemaker-xgboost-2020-06-12-18-38-12-220/output/model.tar.gz'
#s3://sagemaker-us-east-1-328296961357/sagemaker/spark-preprocess-demo/2020-06-12-18-31-52/xgboost_model/sagemaker-xgboost-2020-06-12-18-38-12-220/output/model.tar.gz

's3://sagemaker-us-east-1-328296961357/sagemaker/spark-preprocess-demo/2020-06-12-18-31-52/xgboost_model/sagemaker-xgboost-2020-06-12-18-38-12-220/output/model.tar.gz'

In [11]:
import json
schema = {
    "input": [
        {
            "name": "sex",
            "type": "string"
        }, 
        {
            "name": "length",
            "type": "double"
        }, 
        {
            "name": "diameter",
            "type": "double"
        }, 
        {
            "name": "height",
            "type": "double"
        }, 
        {
            "name": "whole_weight",
            "type": "double"
        }, 
        {
            "name": "shucked_weight",
            "type": "double"
        },
        {
            "name": "viscera_weight",
            "type": "double"
        }, 
        {
            "name": "shell_weight",
            "type": "double"
        }, 
    ],
    "output": 
        {
            "name": "features",
            "type": "double",
            "struct": "vector"
        }
}
schema_json = json.dumps(schema)
print(schema_json)

{"input": [{"name": "sex", "type": "string"}, {"name": "length", "type": "double"}, {"name": "diameter", "type": "double"}, {"name": "height", "type": "double"}, {"name": "whole_weight", "type": "double"}, {"name": "shucked_weight", "type": "double"}, {"name": "viscera_weight", "type": "double"}, {"name": "shell_weight", "type": "double"}], "output": {"name": "features", "type": "double", "struct": "vector"}}


In [25]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.sparkml.model import SparkMLModel

sparkml_data = 's3://{}/{}/{}'.format(bucket, mleap_model_prefix, 'model.tar.gz')
# passing the schema defined above by using an environment variable that sagemaker-sparkml-serving understands
sparkml_model = SparkMLModel(model_data=sparkml_data, env={'SAGEMAKER_SPARKML_SCHEMA' : schema_json})
xgb_model_data = s3_output_location + '/sagemaker-xgboost-2020-06-12-18-38-12-220/output/model.tar.gz'
xgb_model = Model(model_data=xgb_model_data, image=training_image)
#xgb_model = Model(model_data=xgb_model.model_data, image=training_image)

model_name = 'inference-pipeline-' + timestamp_prefix
sm_model = PipelineModel(name=model_name, role=role, models=[sparkml_model, xgb_model])

In [27]:
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

Using already existing model: inference-pipeline-2020-06-12-18-31-52


-----------------!

In [28]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = "F,0.515,0.425,0.14,0.766,0.304,0.1725,0.255"
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker_session, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV, accept=CONTENT_TYPE_CSV)
print(predictor.predict(payload))

b'11.775216102600098'


In [29]:
payload = {"data": ["F",0.515,0.425,0.14,0.766,0.304,0.1725,0.255]}
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload))

b'11.775216102600098'


BATCH TRANSFORM

In [16]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/batch_input_abalone.csv
!printf "\n\nShowing first five lines\n\n"    
!head -n 5 batch_input_abalone.csv 
!printf "\n\nAs we can see, it is identical to the training file apart from the label being absent here.\n\n"  

--2020-06-12 18:50:58--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/batch_input_abalone.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.242.64
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.242.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654 [text/csv]
Saving to: ‘batch_input_abalone.csv.1’

batch_input_abalone 100%[===================>]     654  --.-KB/s    in 0s      

2020-06-12 18:50:58 (16.7 MB/s) - ‘batch_input_abalone.csv.1’ saved [654/654]



Showing first five lines

M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07
F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21
M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155
I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055


As we can see, it is identical to the training file apart from the label being absent here.



In [17]:
batch_input_loc = sagemaker_session.upload_data(path='batch_input_abalone.csv', bucket=bucket, key_prefix='batch')

In [18]:
input_data_path = 's3://{}/{}/{}'.format(bucket, 'batch', 'batch_input_abalone.csv')
output_data_path = 's3://{}/{}/{}'.format(bucket, 'batch_output/abalone', timestamp_prefix)
job_name = 'serial-inference-batch-' + timestamp_prefix
transformer = sagemaker.transformer.Transformer(
    # This was the model created using PipelineModel and it contains feature processing and XGBoost
    model_name = model_name,
    instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    base_transform_job_name='serial-inference-batch',
    sagemaker_session=sagemaker_session,
    accept = CONTENT_TYPE_CSV
)
transformer.transform(data = input_data_path,
                      job_name = job_name,
                      content_type = CONTENT_TYPE_CSV, 
                      split_type = 'Line')
transformer.wait()

........................  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::                  (v2.2)

2020-06-12 18:54:49.105  INFO 8 --- [           main] com.amazonaws.sagemaker.App              : Starting App v2.2 on ef887a1a8771 with PID 8 (/usr/local/lib/sparkml-serving-2.2.jar started by root in /sagemaker-sparkml-model-server)
  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::                  (v2.2)

2020-06-12 18:54:49.105  INFO 8 --- [           main] com.amazonaws.sagemaker.App              : Starting App v2.2 on ef887a1a8771 with PID 8 (/usr/local/li

[2020-06-12:18:55:22:INFO] Determined delimiter of CSV input is ','
169.254.255.130 - - [12/Jun/2020:18:55:22 +0000] "POST /invocations HTTP/1.1" 200 18 "-" "Go-http-client/1.1"
[2020-06-12:18:55:22:INFO] Determined delimiter of CSV input is ','
[2020-06-12:18:55:22:INFO] Determined delimiter of CSV input is ','
169.254.255.130 - - [12/Jun/2020:18:55:22 +0000] "POST /invocations HTTP/1.1" 200 18 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Jun/2020:18:55:22 +0000] "POST /invocations HTTP/1.1" 200 18 "-" "Go-http-client/1.1"
[2020-06-12:18:55:22:INFO] Determined delimiter of CSV input is ','
169.254.255.130 - - [12/Jun/2020:18:55:22 +0000] "POST /invocations HTTP/1.1" 200 17 "-" "Go-http-client/1.1"
[2020-06-12:18:55:22:INFO] Determined delimiter of CSV input is ','
169.254.255.130 - - [12/Jun/2020:18:55:22 +0000] "POST /invocations HTTP/1.1" 200 18 "-" "Go-http-client/1.1"
[2020-06-12:18:55:22:INFO] No GPUs detected (normal if no gpus installed)
[2020-06-12:18:55:22:INFO] Determine

In [19]:
s3 = boto3.resource('s3')

KEY = 'batch_output/abalone/2020-06-11-18-47-41/batch_input_abalone.csv.out'
s3.Bucket(bucket).download_file(KEY, 'batch_output_abalone.csv')

!head batch_output_abalone.csv

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found